This is a code that when given a tif file, processes it and segments the droplets, and saves it as a pyTorch input/output file

In [66]:
import numpy as np
import tifffile as tiff
import cv2
import os
import json

class TIFProcessor:
    def __init__(self, filepath):
        self.filepath = filepath
        self.array = None
        self.gray_arr = None
        self.mask = None
        self.frame_index = None
        self.json_data = []

    def process_all_frames(self, threshold=50, min_consecutive_frames=20):
        for frame_index in range(tiff.imread(self.filepath).shape[0]):
            # TIF file to RGB numpy array
            self.array = tiff.imread(self.filepath, key=frame_index)
            # Convert to grayscale
            self.gray_arr = self.convert_to_grayscale()
            # Crop frame if larger than 1100 pixels
            self.crop_frame()
            # Apply Sobel filter and generate mask
            self.apply_sobel_filter()

            # Calculate mean along the y-axis for the mask
            mask_mean = np.mean(self.mask, axis=0)

            # Identify droplet segments on the x-axis
            droplet_segments_x = self.identify_droplet_segments(mask_mean, threshold, min_consecutive_frames)

            # Remove droplet segments less than 200 pixels long
            droplet_segments_x = [segment for segment in droplet_segments_x if segment[1] - segment[0] >= 350]

            # Further segment on the y-axis
            droplet_segments_y = self.segment_on_y_axis(droplet_segments_x, threshold, min_consecutive_frames)

            # Remove y-axis segments less than 200 pixels long
            droplet_segments_y = [segment for segment in droplet_segments_y if segment[1] - segment[0] >= 350]

            # Accumulate data for JSON
            self.accumulate_json_data(frame_index, droplet_segments_x, droplet_segments_y)

        # Save accumulated JSON data
        self.save_json_data()

    def crop_frame(self, max_height=1100):
        # Crop frame if height is above max_height
        if self.array.shape[0] > max_height:
            self.array = self.array[:max_height, :]

    def convert_to_grayscale(self):
        return np.dot(self.array[..., :3], [0.2989, 0.5870, 0.1140])

    def apply_sobel_filter(self):
        # Apply Sobel filter to the grayscale image
        sobel_x = cv2.Sobel(self.gray_arr, cv2.CV_64F, 1, 0, ksize=3)
        sobel_y = cv2.Sobel(self.gray_arr, cv2.CV_64F, 0, 1, ksize=3)
        self.mask = np.sqrt(sobel_x**2 + sobel_y**2)
        self.frame_index = self.frame_index  # You may want to set the frame_index here

    def identify_droplet_segments(self, signal, threshold, min_consecutive_frames):
        # Identify segments where the signal exceeds the threshold for at least min_consecutive_frames frames
        segments = []
        droplet_started = False
        consecutive_below_threshold = 0

        for i, value in enumerate(signal):
            if value > threshold and not droplet_started:
                droplet_started = True
                start_index = i
                consecutive_below_threshold = 0
            elif value <= threshold and droplet_started:
                consecutive_below_threshold += 1
                if consecutive_below_threshold >= min_consecutive_frames:
                    droplet_started = False
                    end_index = i - min_consecutive_frames
                    segments.append((start_index, end_index))
                    consecutive_below_threshold = 0

        # If a droplet continues to the end of the image, consider it
        if droplet_started:
            end_index = len(signal) - 1
            segments.append((start_index, end_index))

        return segments

    def segment_on_y_axis(self, droplet_segments_x, threshold, min_consecutive_frames):
        # Further segment each droplet segment on the y-axis
        droplet_segments_y = []

        for start_x, end_x in droplet_segments_x:
            # Extract the region of interest from the mask
            droplet_roi = self.mask[:, start_x:end_x + 1]

            # Calculate mean along the x-axis for the droplet ROI
            roi_mean = np.mean(droplet_roi, axis=1)

            # Identify droplet segments on the y-axis
            droplet_segments_y.extend(self.identify_droplet_segments(roi_mean, threshold, min_consecutive_frames))

        return droplet_segments_y

    def accumulate_json_data(self, frame_index, droplet_segments_x, droplet_segments_y):
        # Accumulate data for JSON
        frame_data = {
            'sequence_name': os.path.basename(self.filepath),
            'frame_id': frame_index + 1,
            'droplet_segments_x': droplet_segments_x,
            'droplet_segments_y': droplet_segments_y,
            'image': self.gray_arr.tolist()
        }
        self.json_data.append(frame_data)

    def save_json_data(self):
        # Save accumulated JSON data
        save_path = "../Data/accumulated_data.json"
        with open(save_path, 'w') as json_file:
            json.dump(self.json_data, json_file, indent=2)


# Example of usage
seq42_processor = TIFProcessor("../Data/All_Sequences/4-2.tif")

# Process all frames and save as JSON
seq42_processor.process_all_frames(threshold=50, min_consecutive_frames=20)


TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'
TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'
TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'
TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'
TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'
TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'
TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'
TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'
TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offs

MemoryError: Unable to allocate 16.8 MiB for an array with shape (1100, 2000) and data type float64